In [1]:
#Install pysqlite3 for python and import pandas to use later
#!pip install pysqlite3
from sqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version)
import pandas as pd
from IPython.display import display, HTML

3.45.3


Specify the following queries in SQL on the COMPANY relational database
schema shown in Figure 5.5. Show the result of each query if it is applied to
the COMPANY database in Figure 5.6.
a. Retrieve the names of all employees in department 5 who work more
than 10 hours per week on the ProductX project.
b. List the names of all employees who have a dependent with the same first
name as themselves.
c. Find the names of all employees who are directly supervised by ‘Franklin
Wong’.

In [2]:
dbname = "Company.db"

def printSqlResults(cursor, tblName):
  try:
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    display(HTML("<b><font color=Green> " + tblName + "</font></b>" + df.to_html(index=False)))
  except:
    pass

def runSql(caption, query):
  conn = sqlite3.connect(dbname) # Connect to the database
  cursor = conn.cursor() # Create a cursor (think: it's like a "pointer")
  cursor.execute(query) # Execute the query
  printSqlResults(cursor, caption) # Print the results
  conn.close()

def runSqlWithCommit(caption, query):
  conn = sqlite3.connect(dbname) # Connect to the database
  cursor = conn.cursor() # Create a cursor (think: it's like a "pointer")
  cursor.execute(query) # Execute the query
  printSqlResults(cursor, caption) # Print the results
  conn.commit()
  conn.close()


def runStepByStepSql(query, fromline):
  lines = query.strip().split('\n')
  for lineidx in range(fromline, len(lines)):
    partial_query = '\n'.join(lines[:lineidx])
    caption = 'Query till line:' +  partial_query
    runSql(caption, partial_query + ';')

In [3]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

#Create and Insert Data in EMPLOYEE Table
cursor.execute("""
CREATE TABLE EMPLOYEE(
   Fname     VARCHAR(8) NOT NULL 
  ,Minit     VARCHAR(1) NOT NULL
  ,Lname     VARCHAR(7) NOT NULL
  ,Ssn       INTEGER  NOT NULL PRIMARY KEY
  ,Bdate     DATE  NOT NULL
  ,Address   VARCHAR(24) NOT NULL
  ,Sex       VARCHAR(1) NOT NULL
  ,Salary    INTEGER  NOT NULL
  ,Super_ssn INTEGER 
  ,Dno       INTEGER  NOT NULL
);
""")

cursor.execute("""
INSERT INTO EMPLOYEE(Fname,Minit,Lname,Ssn,Bdate,Address,Sex,Salary,Super_ssn,Dno) VALUES ('John','B','Smith',123456789,'1965-01-09','731 Fondren, Houston, TX','M',30000,333445555,5),
('Franklin','T','Wong',333445555,'1955-12-08','638 Voss, Houston, TX','M',40000,888665555,5),
('Jennifer','J','Zelaya',999887777,'1968-01-19','3321 Castle, Spring, TX','F',25000,987654321,4),
('Alicia','S','Wallace',987654321,'1941-06-20','291 Berry, Bellaire, TX','F',43000,888665555,4),
('Ramesh','K','Narayan',666884444,'1962-09-15','975 Fire Oak, Humble, TX','M',38000,333445555,5),
('Joyce','A','English',453453453,'1972-07-31','5631 Rice, Houston, TX','F',25000,333445555,5),
('Ahmad','V','Jabbar',987987987,'1969-03-29','980 Dallas, Houston, TX','M',25000,987654321,4),
('James','E','Borg',888665555,'1937-11-10','450 Stone, Houston, TX','M',55000,NULL,1);
""")

#Create and Insert Data in DEPARTMENT Table
cursor.execute("""
CREATE TABLE DEPARTMENT(
   Department VARCHAR(14) NOT NULL 
  ,Dnumber     VARCHAR(7) NOT NULL PRIMARY KEY
  ,Mgr_ssn      VARCHAR(9) NOT NULL
  ,Mgr_start_date     VARCHAR(14) NOT NULL
);
""")

cursor.execute("""
INSERT INTO DEPARTMENT(Department,Dnumber,Mgr_ssn,Mgr_start_date) VALUES 
('Research','5','333445555','1988-05-22'),
('Administration','4','987654321','1995-01-01'),
('Headquarters','1','888665555','1981-06-19');
""")

#Create and Insert Data in DEPT_LOCATION Table
cursor.execute("""
CREATE TABLE DEPT_LOCATION(
   Dnumber   INTEGER  NOT NULL
  ,Dlocation VARCHAR(9) NOT NULL
  ,PRIMARY KEY(Dnumber,Dlocation)
);
""")

cursor.execute("""
INSERT INTO DEPT_LOCATION(Dnumber,Dlocation) VALUES (1,'Houston'),
(4,'Stafford'),
(5,'Bellaire'),
(5,'Sugarland'),
(5,'Houston');
""")

#Create and Insert Data in PROJECT Table
cursor.execute("""
CREATE TABLE PROJECT(
   Pname     VARCHAR(15) NOT NULL 
  ,Pnumber   INTEGER  NOT NULL PRIMARY KEY
  ,Plocation VARCHAR(9) NOT NULL
  ,Dnum      INTEGER  NOT NULL
);
""")

cursor.execute("""
INSERT INTO PROJECT(Pname,Pnumber,Plocation,Dnum) VALUES ('ProductX',1,'Bellaire',5),
('ProductY',2,'Sugarland',5),
('ProductZ',3,'Houston',5),
('Computerization',10,'Stafford',4),
('Reorganization',20,'Houston',1),
('Newbenefits',30,'Stafford',4);
""")

#Create and Insert Data in PROJECT Table
cursor.execute("""
CREATE TABLE WORKS_ON(
   Essn  INTEGER  NOT NULL
  ,Pno   INTEGER  NOT NULL
  ,Hours NUMERIC(4,1)
  ,PRIMARY KEY(Essn,Pno)
);
""")

cursor.execute("""
INSERT INTO WORKS_ON(Essn,Pno,Hours) VALUES (123456789,1,32.5),
(123456789,2,7.5),
(666884444,3,40),
(453453453,1,20),
(453453453,2,20),
(333445555,2,10),
(333445555,3,10),
(333445555,10,10),
(333445555,20,10),
(999887777,30,30),
(999887777,10,10),
(987987987,10,35),
(987987987,30,5),
(987654321,30,20),
(987654321,20,15),
(888665555,20,NULL);
""")

cursor.execute("""
CREATE TABLE DEPENDENT(
   Essn           INTEGER  NOT NULL
  ,Dependent_name VARCHAR(9) NOT NULL
  ,Sex            VARCHAR(1) NOT NULL
  ,Bdate          DATE  NOT NULL
  ,Relationship   VARCHAR(8) NOT NULL
  ,PRIMARY KEY(Essn,Dependent_name)
);
""")

cursor.execute("""
INSERT INTO DEPENDENT(Essn,Dependent_name,Sex,Bdate,Relationship) VALUES (333445555,'Alice','F','1986-04-05','Daughter'),
(333445555,'Theodore','M','1983-10-25','Son'),
(333445555,'Joy','F','1958-05-03','Spouse'),
(987654321,'Abner','M','1942-02-28','Spouse'),
(123456789,'Michael','M','1988-01-04','Son'),
(123456789,'Alice','F','1988-12-30','Daughter'),
(123456789,'Elizabeth','F','1967-05-05','Spouse');
""")


conn.commit()
conn.close()


In [4]:
runSql('EMPLOYEE', "select * from EMPLOYEE;")
runSql('DEPARTMENT', "select * from DEPARTMENT;")
runSql('DEPT_Location', "select * from DEPT_Location;")
runSql('PROJECT', "select * from PROJECT;")
runSql('WORKS_ON', "select * from WORKS_ON;")
runSql('DEPENDENT', "select * from DEPENDENT;")

Fname,Minit,Lname,Ssn,Bdate,Address,Sex,Salary,Super_ssn,Dno
John,B,Smith,123456789,1965-01-09,"731 Fondren, Houston, TX",M,30000,333445555.0,5
Franklin,T,Wong,333445555,1955-12-08,"638 Voss, Houston, TX",M,40000,888665555.0,5
Joyce,A,English,453453453,1972-07-31,"5631 Rice, Houston, TX",F,25000,333445555.0,5
Ramesh,K,Narayan,666884444,1962-09-15,"975 Fire Oak, Humble, TX",M,38000,333445555.0,5
James,E,Borg,888665555,1937-11-10,"450 Stone, Houston, TX",M,55000,NaN,1
Alicia,S,Wallace,987654321,1941-06-20,"291 Berry, Bellaire, TX",F,43000,888665555.0,4
Ahmad,V,Jabbar,987987987,1969-03-29,"980 Dallas, Houston, TX",M,25000,987654321.0,4
Jennifer,J,Zelaya,999887777,1968-01-19,"3321 Castle, Spring, TX",F,25000,987654321.0,4


Department,Dnumber,Mgr_ssn,Mgr_start_date
Research,5,333445555,1988-05-22
Administration,4,987654321,1995-01-01
Headquarters,1,888665555,1981-06-19


Dnumber,Dlocation
1,Houston
4,Stafford
5,Bellaire
5,Sugarland
5,Houston


Pname,Pnumber,Plocation,Dnum
ProductX,1,Bellaire,5
ProductY,2,Sugarland,5
ProductZ,3,Houston,5
Computerization,10,Stafford,4
Reorganization,20,Houston,1
Newbenefits,30,Stafford,4


Essn,Pno,Hours
123456789,1,32.5
123456789,2,7.5
666884444,3,40.0
453453453,1,20.0
453453453,2,20.0
333445555,2,10.0
333445555,3,10.0
333445555,10,10.0
333445555,20,10.0
999887777,30,30.0


Essn,Dependent_name,Sex,Bdate,Relationship
333445555,Alice,F,1986-04-05,Daughter
333445555,Theodore,M,1983-10-25,Son
333445555,Joy,F,1958-05-03,Spouse
987654321,Abner,M,1942-02-28,Spouse
123456789,Michael,M,1988-01-04,Son
123456789,Alice,F,1988-12-30,Daughter
123456789,Elizabeth,F,1967-05-05,Spouse
